In [ ]:
addprocs(8)
using AuthorGenderStudy


ERROR (unhandled task failure): InterruptException:
Stacktrace:
 [1] typeinf_code(::Core.MethodInstance, ::Bool, ::Bool, ::Core.Inference.InferenceParams) at ./inference.jl:2584
 [2] typeinf_ext(::Core.MethodInstance, ::UInt64) at ./inference.jl:2622


In [ ]:
using Embeddings
using MLDataUtils


o_embedding_table = load_embeddings(Word2Vec) 

# Add OOV zero vector
const embedding_table = Embeddings.EmbeddingTable(
    [o_embedding_table.embeddings zeros(300)],
    [o_embedding_table.vocab; "<OOV>"]
)

const enc = LabelEnc.NativeLabels("<OOV>", embedding_table.vocab)
to_ind(lbl, enc=enc) = convertlabel(LabelEnc.Indices, lbl , enc)

In [ ]:
const num_test = 100_000
const _data_stream = load_data()
const test_data = collect(Iterators.take(, num_test));
const train_data = _data_stream # Stream has been mutated to remove test_data

In [ ]:
Itera

In [ ]:
train, test = stratifiedobs(first, raw_data, shuffle=true);

In [ ]:
input_rep(datum) = sum(embedding_table.embeddings[:, to_ind(datum.paper_abstract)],2)
input_rep(data::AbstractVector) =  reduce(hcat, input_rep.(data))

In [ ]:
using LIBSVM

In [ ]:
model = svmtrain(input_rep(train), targets(first, train))

In [ ]:
preds, dvs = svmpredict(model, input_rep(test))

In [ ]:
mean(preds .== targets(first, test))

In [ ]:
mean(targets(first, test).==NameToGender.Male)

In [ ]:
1